### Mapping King County Bus Routes, Stops, and Stop Density by Zip Code

In [1]:
import geopandas as gpd
import folium
from folium import plugins
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
bus_routes_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/bus_routes.geojson')
bus_stops_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/bus_stops.geojson')
zip_codes_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/zip_codes.geojson')

In [ ]:
bus_stops_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 22228 entries, 0 to 22227
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   STOP_ID                    22228 non-null  object  
 1   TRANS_LINK_ID              22228 non-null  object  
 2   STOP_TYPE                  22228 non-null  object  
 3   TRF_DISTRICT_CODE          22228 non-null  object  
 4   BEARING_CODE               22228 non-null  object  
 5   CF_CROSS_STREETNAME        22228 non-null  object  
 6   CF_DIST_FROM_INTERSECTION  22228 non-null  float64 
 7   CF_INTERSECTION_LOCCODE    22228 non-null  object  
 8   DISPLACEMENT               22228 non-null  float64 
 9   DIST_FROM_INTERSECTION     22228 non-null  float64 
 10  FARE_ZONE                  22228 non-null  object  
 11  HASTUS_CROSS_STREET_NAME   22228 non-null  object  
 12  INFOSIGN                   22228 non-null  object  
 13  INTERSECTION_LOC       

In [ ]:
bus_routes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ROUTE_ID            148 non-null    object  
 1   LOCAL_EXPRESS_CODE  148 non-null    object  
 2   ROUTE_NUM           148 non-null    object  
 3   SHAPE_Length        148 non-null    float64 
 4   geometry            148 non-null    geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 5.9+ KB


In [ ]:
zip_codes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ZIPCODE            209 non-null    object  
 1   COUNTY             209 non-null    object  
 2   ZIP_TYPE           209 non-null    object  
 3   COUNTY_NAME        209 non-null    object  
 4   CITY_NAME          209 non-null    object  
 5   Shape_Length       209 non-null    float64 
 6   Shape_Area         209 non-null    float64 
 7   ZIPCODE_AREA_SQM   209 non-null    float64 
 8   TOTAL_BUS_STOPS    209 non-null    int64   
 9   BUS_STOPS_PER_SQM  209 non-null    float64 
 10  geometry           209 non-null    geometry
dtypes: float64(4), geometry(1), int64(1), object(5)
memory usage: 18.1+ KB


In [3]:
import folium

def create_kc_map(map_name='base_map', location=[47.608013, -122.335167],
                    zoom_start=12, max_zoom=25,
                    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                    attr='Esri', overlay=False, control=True):

    m = folium.Map(location=location, zoom_start=zoom_start, max_zoom=max_zoom, control_scale=True)
    base_layer = folium.TileLayer(tiles=tiles, attr=attr, name=map_name, overlay=overlay, control=control)
    base_layer.add_to(m)
    return m

In [4]:
m1 = create_kc_map()

Now, we'll add some layers to the map for King County Transit. There is a lot of information, so it would be best if certain features can be toggled on and off.

In [5]:
# Let's visualize the two bus route types independently

express_routes_gdf = bus_routes_gdf[bus_routes_gdf['LOCAL_EXPRESS_CODE'] == 'E']
local_routes_gdf = bus_routes_gdf[bus_routes_gdf['LOCAL_EXPRESS_CODE'] == 'L']

In [6]:
def add_express_routes(m):
    folium.GeoJson(
        express_routes_gdf,
        name='Express Routes',
        style_function=lambda x: {'color': '#32CD32', 'weight': 3, 'opacity': 0.7}
    ).add_to(m)

def add_local_routes(m):
    folium.GeoJson(
        local_routes_gdf,
        name='Local Routes',
        style_function=lambda x: {'color': 'darkred', 'weight': 2, 'opacity': 0.7}
    ).add_to(m)

In [7]:
add_express_routes(m1)
add_local_routes(m1)

In [8]:
# Add legend for route types

def add_route_legend(map_object):
  legend_html = '''
  <div style="position: fixed;
      bottom: 50px; left: 50px; width: 150px; height: 80px;
      border:2px solid grey; z-index:9999; font-size:14px;
      ">&nbsp; <b>Route Type:</b><br>
      &nbsp; Express &nbsp; <i style="background:brightgreen; width:20px; height:10px; display:inline-block;"></i><br>
      &nbsp; Local &nbsp; <i style="background:darkred; width:20px; height:10px; display:inline-block;"></i>
  </div>
  '''

  legend = folium.Html(legend_html, script=True)
  folium.Element(legend.render()).add_to(map_object)


In [10]:
add_route_legend(m1)

In [11]:
#m1

Output hidden; open in https://colab.research.google.com to view.

Now we'll add bus stop information.

In [12]:
from folium.plugins import MarkerCluster

def mark_bus_stops(map_object, bus_stops_gdf):
  for _, row in bus_stops_gdf.iterrows():
    folium.Marker(
        location=[row['YCOORD'], row['XCOORD']],
        icon=folium.Icon(icon='bus', prefix='fa', color='orange')
    ).add_to(map_object)

In [13]:
from folium.plugins import HeatMap

def create_heatmap_bus_stops(map_object, bus_stops_gdf):
  heatmap_data = bus_stops_gdf[['YCOORD', 'XCOORD']].values.tolist()
  HeatMap(heatmap_data).add_to(map_object)

In [14]:
mark_bus_stops(m1, bus_stops_gdf)

create_heatmap_bus_stops(m1, bus_stops_gdf)

In [15]:
#m1

Output hidden; open in https://colab.research.google.com to view.

In [20]:
pip install mapbox-tilesets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.19.2
    Uninstalling jsonschema-4.19.2:
      Successfully uninstalled jsonschema-4.19.2


In [22]:
pip install mapbox-tilesets[estimate-area]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 58.8 MB/s eta 0:00:00


Let's add ZIPCODE information.

In [16]:
import folium

def display_zipcode_overlay(map_object, zip_codes_gdf, attribute='TOTAL_BUS_STOPS'):

    # Create a tooltip that dynamically fetches the ZIPCODE and the specified attribute
    folium.GeoJson(
        zip_codes_gdf,
        style_function=lambda x: {'color': 'blue', 'weight': 2, 'opacity': 0.5},
        tooltip=folium.GeoJsonTooltip(fields=['ZIPCODE', attribute],
                                      aliases=['ZIP Code:', f'{attribute}:'],
                                      localize=True)
    ).add_to(map_object)


In [17]:
display_zipcode_overlay(m1, zip_codes_gdf)

In [18]:
folium.LayerControl().add_to(m1)

In [19]:
m1

Output hidden; open in https://colab.research.google.com to view.